In [ ]:
%pip install azure-ai-evaluation
%pip install openpyxl

In [8]:
# Import libraries
import os
import sys
import json
import time
import pandas as pd
from dotenv import load_dotenv, find_dotenv
from openai import AzureOpenAI
from azure.ai.evaluation import GroundednessProEvaluator, GroundednessEvaluator, QAEvaluator, SimilarityEvaluator

sys.path.append('..')
from common_utils import *

# Load Azure OpenAI and AI Search variables and create clients
openai_config, ai_search_config = load_config()

# Initialzing Groundedness and Similarity evaluators
model_config = {
    "azure_endpoint": openai_config["aoai_endpoint"],
    "api_key": openai_config["aoai_key"],
    "azure_deployment": openai_config["aoai_deployment_name"],
    "api_version": openai_config["api_version"]
}
groundedness_eval = GroundednessEvaluator(model_config)
similarity_eval = SimilarityEvaluator(model_config)

aoai_endpoint: https://openai-asc-swit-north.openai.azure.com/
aoai_deployment_name: gpt-4o
oai_embedding_model: ada
aoai_rerank_model: gpt-4o-mini
ai_search_index_name_regs: rag-index-regs
ai_search_index_name_docs: rag-index-docs


### Test with one question the end-to-end process:
- Generate query for AI Search
- Hybrid search with Semantic ranker
- Filter chunks comparing with the question
- Generate the answer with the relevant chunks as context
- Evaluate the answer compared with the expected answer

In [10]:
# Test with one question
question = "What is included in my Northwind Health Plus plan?"
print(f'QUESTION REWRITE: {question}')
query = generate_query(openai_config["openai_client"],
                       openai_config["aoai_deployment_name"],
                       question)
print(f'Query REWRITTEN: {query}')

# Hybrid search
results, num_results = semantic_hybrid_search(ai_search_config["ai_search_client_docs"],
                                              openai_config["openai_client"],
                                              openai_config["aoai_embedding_model"],
                                              query, 10)
print(f"query: {query}, num results: {num_results}")
show_results(results, query)

# Valid chunks for the user question
valid_chunks, num_chunks = get_filtered_chunks(openai_config["openai_client"],
                                               openai_config["aoai_rerank_model"],
                                               results, question)

# Generate answer:
answer = generate_answer(openai_config["openai_client"],
                         openai_config["aoai_deployment_name"],
                         valid_chunks, query)
print(f"\n>> Answer: {answer}")

# Evaluate answer
expected_answer = """The Northwind Health Plus plan includes the following benefits coverage:
- Deductible: $2,000 per year.
- Coinsurance: 20% of the cost of a covered service after the deductible is met.
- Out-of-Pocket Maximum: $4,000 per year, including deductible, coinsurance, and copayments.
- In-Network Provider: Lower copayments and coinsurance amounts.
- Out-of-Network Provider: Higher copayments and coinsurance amounts.
- Preventive Care: Covered at 100% with no copayment, deductible, or coinsurance.
- Prescription Drugs: Subject to a copayment, varying by drug type. Generic drugs usually have lower copayments.
- Mental Health and Substance Abuse Services: Subject to a copayment and deductible, varying by service type."""

groundedness_score, similarity_score = evaluate_answer(groundedness_eval, similarity_eval, query, valid_chunks, answer, expected_answer)
print(f'groundedness_score: {groundedness_score}')
print(f'similarity_score: {similarity_score}')

QUESTION REWRITE: What is included in my Northwind Health Plus plan?
Query REWRITTEN: Northwind Health Plus plan benefits coverage details
query: Northwind Health Plus plan benefits coverage details, num results: 123
Hybrid Search Results: [
  {
    "id": "142",
    "title": "Northwind_Health_Plus_Benefits_Details.md",
    "content": ",000 per year.\n\nCoinsurance: Coinsurance is the percentage of the cost of a covered service that an\nemployee must pay after the deductible is met. Northwind Health Plus has a coinsurance of\n20%.\n\nOut-of-Pocket Maximum: The out-of-pocket maximum is the maximum amount of money\nthat an employee has to pay for covered services in a plan year. This amount includes the\ndeductible, coinsurance, and copayments. Northwind Health Plus has an out-of-pocket\nmaximum of $4,000 per year.\n\nIn-Network Provider: An in-network provider is a health care provider or facility that is\ncontracted with the insurance company. Employees who use an in-network provider wi

In [11]:
# Test with all questions in an excel file
input_file = "ground_truth.xlsx"
df = pd.read_excel(input_file)
data_dict = df.to_dict(orient='records')

# For earch line in the input Excel file
for i, line in enumerate(data_dict):
    question = line['QUESTION']
    print(f"[{i+1}] question: {question}")
    query = generate_query(openai_config["openai_client"],
                        openai_config["aoai_deployment_name"],
                        question)
    print(f'\tQuery: {query}')

    # Hybrid search
    results, num_results = semantic_hybrid_search(ai_search_config["ai_search_client_docs"],
                                                openai_config["openai_client"],
                                                openai_config["aoai_embedding_model"],
                                                query, 10)
    print(f"\tnum results: {num_results}")
    #show_results(results, query)

    # Valid chunks for the user question
    valid_chunks, num_chunks = get_filtered_chunks(openai_config["openai_client"],
                                                openai_config["aoai_rerank_model"],
                                                results, question)
    print(f"\tnum valid chunks: {num_chunks}")

    # Generate answer:
    answer = generate_answer(openai_config["openai_client"],
                            openai_config["aoai_deployment_name"],
                            valid_chunks, query)
    print(f"\n>> Answer: {answer}")

    # Evaluate answer
    expected_answer = "Your Northwind Health Plus plan includes coverage for medical, vision, and dental services. It also provides coverage for prescription drugs, mental health and substance abuse services, and preventive care. You can choose from a variety of in-network providers, including primary care physicians, specialists, hospitals, and pharmacies. Emergency services are covered, both in-network and out-of-network. Co-pays, deductibles, and out-of-pocket maximums may apply to your plan. Your plan may also include separate deductibles for different services, such as prescription drugs and hospitalization. It is important to know what your plan covers and what the cost-sharing requirements are"
    groundedness_score, similarity_score = evaluate_answer(groundedness_eval, similarity_eval, query, valid_chunks, answer, expected_answer)
    print(f'groundedness_score: {groundedness_score}')
    print(f'similarity_score: {similarity_score}')
    print('--------------------------------------------------')

[1] question: What is included in my Northwind Health Plus plan?
	Query: Northwind Health Plus plan benefits coverage details
	num results: 123
	num valid chunks: 2

>> Answer: The Northwind Health Plus plan includes the following coverage details:

- **Copayment**: A fixed amount paid for a covered service at the time of service.
- **Deductible**: $2,000 per year, which is the amount paid out-of-pocket before the plan covers services.
- **Coinsurance**: 20% of the cost of a covered service after meeting the deductible.
- **Out-of-Pocket Maximum**: $4,000 per year, including deductible, coinsurance, and copayments.

**Provider Types**:
- **In-Network Provider**: Lower copayments and coinsurance.
- **Out-of-Network Provider**: Higher copayments and coinsurance.

**Exceptions**:
- **Preventive Care**: Covered at 100% with no copayment, deductible, or coinsurance.
- **Prescription Drugs**: Subject to a copayment, varying by drug type.
- **Mental Health and Substance Abuse Services**: Subj